# Pckage List

In [1]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from haversine import haversine, Unit

/home/ubuntu2204/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Load

In [2]:
df = pd.read_csv("../data/prep3.csv", encoding='cp949')

# Function

## API 키 설정

In [ ]:
'../../../api_key'

In [6]:
key = open('../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

In [18]:
class OllamaSentenceTransformer():
    def __init__(
            self,
            *args,
            **kargs,
            ) -> None:
                # self.base_url = kargs.get("base_url", "http://localhost:11434")
                self.model = kargs.get("model","bge-m3")
                self.embedding_model = OllamaEmbeddings(
                            model = self.model,
                            base_url = base_url,
                        )
                        
    
    def encode(
            self,
            documents:Union[str, List[str], np.ndarray],
            *args,
            **kargs,
        )-> np.ndarray:
        if isinstance(documents, str):
            document_embeddings = self.embedding_model.embed_query(documents)
            return np.array(document_embeddings)
        
        if isinstance(documents, np.ndarray):
            documents = documents.tolist()
        
        document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
        return np.array(document_embeddings)
        


In [8]:
sentence_transformer = OllamaSentenceTransformer(base_url=base_url)

## 위치 관련 정보 제공

In [9]:
def get_lat_lon(location_name):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'json'
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ChatGPT-Example/1.0; +http://yourdomain.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    # 응답 확인
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        print("응답 내용:", response.text)
        return None

    try:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            print("위치 정보를 찾을 수 없습니다.")
            return None
    except ValueError as e:
        print("JSON 파싱 오류:", e)
        print("응답 내용:", response.text)
        return None

# 테스트
location = '진천군'
result = get_lat_lon(location)
print(f"{location} 위도/경도: {result}")


진천군 위도/경도: (36.855461, 127.4353927)


# 전체 툴 인코딩 값

In [10]:
with open('../data/encoded_tool_name2.pkl', "rb") as f:
    all_tool_encoded_array = pickle.load(f)

# Input Content

In [11]:
with open('../data/user_input_content.json', "rb") as f:
    input_content = json.load(f)

In [19]:
def run_search_tool(inputed_content, df, all_tool_encoded_array,head_count=3):
    spot_location = get_lat_lon(inputed_content['location'])
    if not spot_location:
        print("위치 정보를 가져오지 못했습니다.")
        return None
    
    encoded_tool_name = sentence_transformer.encode(inputed_content['tool_name'])
    
    df['tool_sim_result'] = cosine_similarity(all_tool_encoded_array, encoded_tool_name.reshape(1, -1)).flatten()
    
    df_s = df[df['tool_sim_result'] > 0.9]
    # use_col = ['대여소명', '위도', '경도', '주소', 'tool_sim_result']
    use_col = ['공구 이름', '과금기준', '수량','대여장소명', '상세주소','전화번호', '평일오픈시간', '평일클로즈시간', '생성일시', '요금']

    
    df_s['Distance(Km)'] = df_s[["위도", "경도"]].apply(lambda x: haversine((spot_location[0], spot_location[1]), (x['위도'], x['경도']), unit=Unit.KILOMETERS), axis=1)
    df_s.sort_values(by='Distance(Km)', ascending=True, inplace=True)
    
    return df_s[use_col].head(head_count)

In [17]:
input_content


{'tool_name': '전동드릴',
 'location': '강남역',
 'job_content': '전동드릴을 사용해서 나무를 고정하고 싶어'}

In [20]:
df_filter = run_search_tool(input_content, df, all_tool_encoded_array)

KeyboardInterrupt: 

# Question

In [ ]:
question = f"""나는 {input_content['tool_name']}을(를) 빌리고 싶어. 
그리고 이 도구를 이용해서 하려는 작업은 '{input_content['job_content']}' 이야. 
해당 작업을 하면서 같이 빌리면 좋은 공구도 함께 알려줘"""

# Ver.ChatGPT

In [ ]:
def invoke(question, info = None, api_key=api_key, model="gpt-4o", temperature=0.7):
    client = OpenAI(api_key=api_key)
    if info is not None:
        system_content = (
            f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
                서울시에서 운영하는 대여 공구 찾기 정보 시스템이야.
                관련 정보는 다음과 같아: {info}
                이 정보를 참고해서 사용자의 질문에 친절히 답해줘."""
        )
    else:
        system_content = (
            """너는 사람들의 질문에 친절히 답해주는 도우미야.
                서울시에서 운영하는 대여 공구 찾기 정보 시스템이야.
                사용자의 질문에 친절히 답해줘."""
        )

    # 메시지 구성
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": question}
    ]
    # 스트리밍 요청
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=True  # ⭐ 핵심 옵션!
    )

    # generator 반환 (chunk 단위 텍스트 출력)
    return stream
    

In [46]:
display("가까운 대여소 : ", df_filter)
for chunk in invoke(question):
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end='', flush=True)

'가까운 대여소 : '

,공구 이름,과금기준,수량,대여장소명,상세주소,전화번호,평일오픈시간,평일클로즈시간,생성일시,요금
10414,전동드릴,1일,3,서초1동반딧불센터,서울특별시 서초구 남부순환로339길 47-1 2층,02-521-0364,10:00,18:00,2018-01-23 17:33:00,10000원
10423,전동드릴,1일,1,반포반딧불센터,서울특별시 서초구 신반포로42길 12 1층,02-516-7887,10:00,18:00,2018-01-23 17:33:00,10000원
10306,전동드릴,없음,1,잠원동주민센터,나루터로 38,02-2155-7543,09:00,18:00,2023-08-02 07:49:01,무료


전동드릴을 사용해서 나무를 고정하려면, 다음과 같은 공구를 함께 대여하는 게 좋습니다:

1. **드릴 비트 세트**: 나무에 구멍을 뚫기 위한 다양한 크기의 드릴 비트가 필요합니다.
2. **스크루**: 나무를 고정하기 위한 적절한 크기와 길이의 스크루가 필요합니다.
3. **스크루드라이버**: 드릴로 스크루를 박기 어려운 작은 공간에서는 스크루드라이버가 유용할 수 있습니다.
4. **클램프**: 작업 중 나무를 고정시켜 안정성을 높여줄 수 있는 클램프가 있으면 좋습니다.
5. **수평기**: 나무를 수평으로 고정하기 위해 수평기를 사용하면 작업의 정확성을 높일 수 있습니다.
6. **줄자**: 정확한 측정을 위해 필수적입니다.
7. **연필 및 표시 도구**: 정확한 위치를 표시하는 데 필요합니다.

서울시에서는 주민들이 다양한 공구를 대여할 수 있는 공구 대여소를 운영하고 있습니다. 가까운 주민센터에 문의하시거나 서울시 공구 대여소 웹사이트를 통해 대여 가능 여부와 절차를 확인해보세요. 안전하게 작업하시길 바랍니다!

In [ ]:
pd.read_csv('')

# Ver. Gemma3:12b

In [ ]:
from langchain_ollama.chat_models import ChatOllama

def invoke(question, info = None, model="gemma3:12b", temperature=0.7):
    chat = ChatOllama(model=model,
                      base_url=base_url,
                      temperature=temperature,)
    # 시스템 메시지 조건 처리
    if info is not None:
        system_content = (
            f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
                서울시에서 운영하는 대여 공구 찾기 정보 시스템이야.
                관련 정보는 다음과 같아: {info}
                이 정보를 참고해서 사용자의 질문에 친절히 답해줘."""
        )
    else:
        system_content = (
            """너는 사람들의 질문에 친절히 답해주는 도우미야.
                서울시에서 운영하는 대여 공구 찾기 정보 시스템이야.
                사용자의 질문에 친절히 답해줘."""
        )

    # 메시지 구성
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": question}
    ]

    # invoke로 전체 응답 받기
    # response = chat.invoke(messages).content
    response = chat.stream(messages)

    # 응답 내용 반환
    return response


In [14]:
display("가까운 대여소 : ", df_filter)
for chunk in invoke(question):
    print(chunk.content, end='', flush=True)

'가까운 대여소 : '

,공구 이름,과금기준,수량,대여장소명,상세주소,전화번호,평일오픈시간,평일클로즈시간,생성일시,요금
10414,전동드릴,1일,3,서초1동반딧불센터,서울특별시 서초구 남부순환로339길 47-1 2층,02-521-0364,10:00,18:00,2018-01-23 17:33:00,10000원
10423,전동드릴,1일,1,반포반딧불센터,서울특별시 서초구 신반포로42길 12 1층,02-516-7887,10:00,18:00,2018-01-23 17:33:00,10000원
10306,전동드릴,없음,1,잠원동주민센터,나루터로 38,02-2155-7543,09:00,18:00,2023-08-02 07:49:01,무료


안녕하세요! 전동드릴을 찾으시는군요. 나무를 고정하는 작업에 필요한 전동드릴과 함께 사용하면 좋을 공구들을 찾아드릴게요.

**1. 전동드릴 대여 정보**

서울시에서는 공유소가 운영하는 공구 대여 서비스를 통해 전동드릴을 빌릴 수 있습니다.

*   **공유소 위치:** 서울시 곳곳에 위치하고 있으며, 가까운 공유소를 찾으려면 서울시 공유소 홈페이지에서 검색할 수 있습니다.
    *   **서울시 공유소 홈페이지:** [https://www.seoulshare.kr/](https://www.seoulshare.kr/)
*   **대여 조건:**
    *   서울시민이라면 누구나 이용 가능합니다.
    *   대여료는 공구 종류 및 대여 기간에 따라 다릅니다.
    *   대여 전 안전 교육 이수 또는 온라인 교육 수료가 필요할 수 있습니다.
*   **예약 방법:** 서울시 공유소 홈페이지 또는 앱에서 예약할 수 있습니다.

**2. 함께 빌리면 좋은 공구**

나무를 고정하는 작업에 전동드릴과 함께 사용하면 다음과 같은 공구들이 유용합니다.

*   **드라이버:** 전동드릴에 맞는 드라이버 비트를 사용하여 나사를 조일 때 사용합니다.
*   **나사:** 나무를 고정할 때 필요한 나사를 준비합니다. 나무의 종류와 크기에 맞는 나사를 선택하는 것이 중요합니다.
*   **줄자:** 나무를 정확하게 측정하고 자를 때 사용합니다.
*   **연필:** 나무에 표시를 할 때 사용합니다.
*   **보안경:** 작업 중 파편이 튀는 것을 보호하기 위해 착용합니다.
*   **장갑:** 작업 중 손을 보호하기 위해 착용합니다.

**3. 추가 조언**

*   작업 전에 전동드릴 사용법을 숙지하고 안전 수칙을 준수하세요.
*   나무의 종류와 크기에 맞는 전동드릴과 나사를 선택하세요.
*   작업 중에는 항상 안전에 유의하세요.

**4. 추가 문의**

*   더 궁금한 점이 있으시면 언제든지 다시 질문해주세요.
*   서울시 공유소 홈페이지 또는 고객센터

In [ ]:
def invoke(question, 
           prompt,
           api_key=api_key,
           model="gpt-4o",
           temperature=0.7):
    client = OpenAI(api_key=api_key)

    # 메시지 구성
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": question}
    ]
    # 스트리밍 요청
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        # stream=True  # ⭐ 핵심 옵션!
    )

    # generator 반환 (chunk 단위 텍스트 출력)
    return stream
    

In [21]:
def invoke(
           prompt,
           api_key=api_key,
           model="gpt-4o",
           temperature=0.7):
    client = OpenAI(api_key=api_key)

    # 메시지 구성
    messages = [
        # {"role": "system", "content": prompt},
        {"role": "user", "content": prompt}
    ]
    # 스트리밍 요청
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        # stream=True  # ⭐ 핵심 옵션!
    )

    # generator 반환 (chunk 단위 텍스트 출력)
    return stream

In [22]:
prompt = f"""
                너는 가위바위보 천재야 가위바위보 이기는 방법에 대해서 설명해줘
                """

In [23]:
response = invoke(prompt)

In [32]:
response.choices[0].message.content

'가위바위보는 운에 크게 좌우되는 게임이지만, 몇 가지 전략을 통해 이길 확률을 조금 높일 수 있습니다. 다음은 가위바위보에서 이길 가능성을 증가시키는 몇 가지 팁입니다.\n\n1. **상대방의 패턴 파악하기**: 상대방이 이전에 어떤 선택을 했는지 관찰하고, 반복되는 패턴이 있는지 확인하세요. 사람들은 종종 무의식적으로 패턴을 따르기도 합니다.\n\n2. **심리전 활용하기**: 상대방이 여러분이 어떤 선택을 할 것이라고 생각할지를 고려하고, 그에 반하는 선택을 하세요. 예를 들어, 상대방이 당신이 가위를 낼 것이라고 생각한다면 바위를 내 보세요.\n\n3. **무작위 선택**: 상대방이 당신의 패턴을 읽지 못하게 하려면, 완전히 무작위로 선택하는 것이 좋습니다. 이를 통해 예측 가능성을 줄일 수 있습니다.\n\n4. **초반에 바위 내기**: 통계적으로 많은 사람들이 처음에 가위나 보다는 바위를 내는 경향이 있습니다. 따라서 첫 판에 바위를 내는 것이 유리할 수도 있습니다.\n\n5. **심리적 유도**: 상대방에게 특정 선택을 하도록 유도할 수 있습니다. 예를 들어, 가위를 낼 것처럼 손을 움찔하는 등의 동작으로 상대방을 혼란스럽게 할 수 있습니다.\n\n이러한 방법들은 확률을 조금 높일 수는 있지만, 가위바위보는 여전히 운이 크게 작용하는 게임임을 기억하세요.'